In [1]:
import os
import cv2
import glob2
import pydicom
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import exposure
import pickle
from mask_functions import *

In [2]:
sz = 512
sz0 = 1024
PATH_TRAIN0 = '../Data/dicom-images-train/'
PATH_TRAIN1 = '../Data/dicom-images-test/'
PATH_TEST = '../Data/stage_2_images/'

In [3]:
train = glob2.glob(os.path.join(PATH_TRAIN0, '**/*.dcm'))+glob2.glob(os.path.join(PATH_TRAIN1, '**/*.dcm'))
test = glob2.glob(os.path.join(PATH_TEST, '**/*.dcm'))

df = pd.read_csv('../Data/stage_2_train.csv').set_index('ImageId')
idxs = set(df.index)
train_names = []
for f in train: #remove images without labels
    name = f.split('/')[-1][:-4]
    if name in idxs: train_names.append(f)

In [4]:
def convert_image(filename, sz=1024):
    ds = pydicom.read_file(str(filename))
    img = ds.pixel_array
    if sz != 1024:
        img = cv2.resize(img, (sz, sz))
    img = exposure.equalize_adapthist(img) # contrast correction
    return img.astype(np.float32)

In [5]:
len(train_names)

12047

In [6]:
with open(r"../Data/sample_index.pickle", "rb") as input_file:
    index = pickle.load(input_file)

In [7]:
imgs_torch = np.stack([convert_image(fname,sz) for fname in train_names],0)
imgs_torch = imgs_torch[index]

with open(r"../Data/train_imgs_torch_"+str(sz)+".pickle", "wb") as output_file:
    pickle.dump(imgs_torch, output_file)

OverflowError: cannot serialize a bytes object larger than 4 GiB

In [ ]:
# # chunk 2
# imgs_torch = np.stack([convert_image(fname,sz) for fname in train_names[3000:6000]],0)[...,None]
# with open(r"../Data/train_imgs_torch_"+str(sz)+"_c2.pickle", "wb") as output_file:
#     pickle.dump(imgs_torch, output_file)

In [ ]:
len(test)

In [ ]:
imgs_torch = np.stack([convert_image(fname,sz) for fname in test],0)
with open(r"../Data/test_imgs_torch_"+str(sz)+".pickle", "wb") as output_file:
    pickle.dump(imgs_torch, output_file)

In [ ]:
def convert_mask(fname, sz=1024):
    masks = df.loc[fname.split('/')[-1][:-4],'EncodedPixels']
    img = np.zeros((sz0,sz0))
    #do conversion if mask is not " -1"
    if(type(masks) != str or (type(masks) == str and masks != '-1')):
        if(type(masks) == str): masks = [masks]
        else: masks = masks.tolist()
        for mask in masks:
            img += rle2mask(mask, sz0, sz0).T
    if sz != 1024:
        img = cv2.resize(img, (sz, sz))
    return img.astype(np.float32)/255

In [ ]:
imgs_torch = np.stack([convert_mask(fname,sz) for fname in train_names],0)
imgs_torch = imgs_torch[index]

with open(r"../Data/masks_torch_"+str(sz)+".pickle", "wb") as output_file:
    pickle.dump(imgs_torch, output_file)